# RFM и исторические данные маркетинговых кампаний


 Частью нашего сервиса  является оценка не только метрик из независимых источников, но и оценка собственных исторических данных для помощи в принятии решения по будущему запуску рекламных кампаний. Мы выдвинули гипотезу, что аудитория пабликов вк соответствует (с возможным сдвигом в возрасте) аудитории их первичных источников. Например, если мы хотим оценить потенциальную аудиторию сайта РБК на интерес к какому-то продукту и их доходность, мы можем сделать тестовую рекламную кампанию или, если они есть, - взять исторические данные предыдущих рекламных кампаний в вк на аудитории группы рбк (мы можем узнать, из какой группы пришел клиент по динамическим utm меткам).Таргетированная рекламная кампания в вк намного дешевле рекламы на других платформах и мы знаем параметры настройки на целевую аудиторию - пол, город и интересы нам уже известны из наших данных и описанию паблика. Далее мы можем собрать данные и проанализировать спрос на тот или иной продукт. В том числе, мы можем отслеживать дальнейшие действия покупателей, пришедших из рекламной кампании, оценивать их финансовые показатели (интерепретировать как доходность) и сегментировать их, напрмиер, методом RFM.  

Модель RFM предназначена для сегментации пользователей на основе трех параметров: 
1.   **recency** (как давно клиент использовал продукт); 
2.   **frequency** (как часто клиент пользуется продуктом); 
3.   **monetary** (сколько денег принес клиент бизнесу/какой денежный оборот и пользователя). 

Мы ранжируем эти параметры относительно общего распределения всех данных и сегменируем по заранее заданным сегментам: **ABOUT TO SLEEP, RECENT CUSTOMER, CANNOT LOOSE THEM, CHAMPION, LOST CUSTOMER, LOYAL, POTENTIAL LOYALIST, AVERAGE**.

(Для оценки RFM нам понадобилось добавить один столбец в виде денежного оборота у клиента на карте в изначальную таблицу транзакций). 

Таким образом, мы можем как протестировать рекламную кампанию на похожем сегменте пользователей, так и понимать, какой сегмент пользователей из разных каналов к нам приходит. Например, много средних, не очень активных пользователей или не так много золотых.


In [1]:
import plotly.express as px

def draw_plotly_historic_data_returns(data_plot: pd.DataFrame):
  """ 
  Функция предзназначена для визуализации доходов со всех каналов
    Parameters
        ----------
        **kwargs : pd.DataFrame
  """
      fig = px.bar(data_plot.groupby(by='source').sum(), 
                   x = list(data_plot.groupby(by='source').sum().index), 
                   y = ['contract_sum'])
      fig.update_layout(
      title="Итоговая сумма контрактов по каналам",
      xaxis_title="source",
      yaxis_title="contract_sum_all")
      return fig


def draw_plotly_historic_data(data_plot: pd.DataFrame):
    """
    Функция предзназначена для визуализации распределения интересов к продуктам у пользователей, перешедших по ссылке (купивших и не купивших)
       Parameters
        ----------
        **kwargs : pd.DataFrame
  """
      fig = px.bar(data_plot.groupby(by='source').mean(), 
                   y = list(data_plot.groupby(by='source').mean().index), 
                   x=data_plot.columns[-5:])
      fig.update_layout(
      title="Распределение заказанного продукта по каналам",
      xaxis_title="Интерес к продукту (процент покупок от общего количества перешедших)",
      yaxis_title="Каналы")
      return fig

def draw_plotly_historic_data_gender(data_plot: pd.DataFrame):
    """
    Функция предзназначена для визуализации распределения пола заинтересованных пользователей по каждому каналу
       Parameters
        ----------
        **kwargs : pd.DataFrame
  """

      fig = px.bar(data_plot.groupby(by='source').mean(), 
                   y = list(data_plot.groupby(by='source').mean().index), 
                   x=['gender'])
      fig.update_layout(
      title="Распределение М/Ж по каналам",
      xaxis_title="Пол",
      yaxis_title="Каналы")
      return fig

from plotly.subplots import make_subplots
import plotly.graph_objects as go

def rfm_segments_compaign(data_rfm, data_marketing_compaign):
      """
    Функция предзназначена для визуализации сегментации на основе RFM модели по истечению тестового срока клиентов, купивших продукт во время рекламной кампании
       Parameters
        ----------
        **kwargs : pd.DataFrame, pd.DataFrame
  """

    def rfm_query(adid):
      ans = data_rfm[
                     data_rfm['client_id']
                     .isin(data_marketing_compaign[
                                         ((data_marketing_compaign.is_deal == 1) & (data_marketing_compaign.adid == adid))
                                         ]
                           ['client_id']
                                        .unique())
                     ].rfm_score_name.value_counts()
      return ans
      
    data1 = rfm_query('8730nd')
    data2 = rfm_query('873kdb')
    data3 = rfm_query('n27cl3')

    fig = make_subplots(rows=1, cols=3, 
                        specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]], 
                                              subplot_titles=['id - 8730nd', 'id - 873kdb', 'id - n27cl3'])
    fig.add_trace(
        go.Pie(values = list(data1), labels=list(data1.index)),row=1, col=1)
    fig.add_trace(
        go.Pie(values = list(data2), labels=list(data2.index)),row=1, col=2)
    fig.add_trace(
        go.Pie(values = list(data3), labels=list(data3.index)),row=1, col=3)
    fig.update_layout(height=600, width=800, title_text="Сегмент пользователей по RFM, пришедших из рекламных кампаний")
    return fig

IndentationError: ignored